# SpanBERTa: Reconocimiento de Entidades Nombradas con Transformers
##https://github.com/chriskhanhtran/spanish-bert

# Introducción

SpanBERTa es un modelo de lenguaje basado en tecnología Transformer para español, entrenado desde cero sobre un gran corpus. En esta notebook, llevaremos a cabo un proceso de fine-tuning sobre SpanBERTa para la tarea de Reconocimiento de Entidades Nombradas.

Utilizamos el script `run_ner.py` (dentro de la carpeta drive) y el dataset Güemes Documentado para el fine-tuning.

# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
CARPETA_COLAB = '/content/drive/MyDrive/ProyectoNERC/NERC/'

Mounted at /content/drive


Instalamos paquetes e importamos.

In [2]:
!pip install transformers
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 36.9 MB/s 
     |████████████████████████████████| 6.6 MB 38.2 MB/s 
     |████████████████████████████████| 86 kB 7.9 MB/s 
     |████████████████████████████████| 596 kB 53.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.6 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=853a3ec6b2158527862b7bce312d795fc337ee57a1b67451e5c375d723fc7cd5
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [3]:
import transformers
import random

# Datos


## 1. Cargamos Dataset

In [4]:
tokens = []
with open(CARPETA_COLAB+'dataset/GD-1.conll', 'r', encoding="utf-8") as f:
  for line in f.readlines():
    splitted_text = line.split()
    if len(splitted_text) == 3:
      tokens.append((splitted_text[0], splitted_text[2]))

Train y test (a y b) para Güemes Documentado
   - testa: Datos de prueba en español para la fase development
   - testb: Datos de prueba final
   - train: Datos de entrenamiento

In [5]:
# Division 60 - 20 - 20. Se realiza a mano para asegurar que cada set termine en una oracion
train = tokens[:31020]
testa = tokens[31020:41395]
testb = tokens[41395:]

In [6]:
with open("train_temp.txt", 'w', encoding="utf-8") as f:
  for token in train:
    f.write(token[0] + ' ' + token[1] + '\n')

with open("dev_temp.txt", 'w', encoding="utf-8") as f:
  for token in testa:
    f.write(token[0] + ' ' + token[1] + '\n')

with open("test_temp.txt", 'w', encoding="utf-8") as f:
  for token in testb:
    f.write(token[0] + ' ' + token[1] + '\n')

## 2. Preprocesamiento

In [7]:
MAX_LENGTH = 120 #@param {type: "integer"}
MODEL = "dccuchile/bert-base-spanish-wwm-cased" #@param ["chriskhanhtran/spanberta", "bert-base-multilingual-cased", "dccuchile/bert-base-spanish-wwm-cased"]

Acortamos oraciones a un máximo (`MAX_LENGTH`) en cantidad de tokens.

In [8]:
#%%capture
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

--2022-06-10 15:31:47--  https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 991 [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      

2022-06-10 15:31:47 (51.3 MB/s) - ‘preprocess.py’ saved [991/991]



In [9]:
!python3 preprocess.py train_temp.txt $MODEL $MAX_LENGTH > train.txt
!python3 preprocess.py dev_temp.txt $MODEL $MAX_LENGTH > dev.txt
!python3 preprocess.py test_temp.txt $MODEL $MAX_LENGTH > test.txt

Downloading: 100% 364/364 [00:00<00:00, 267kB/s]
Downloading: 100% 648/648 [00:00<00:00, 525kB/s]
Downloading: 100% 236k/236k [00:00<00:00, 8.37MB/s]
Downloading: 100% 469k/469k [00:00<00:00, 16.6MB/s]
Downloading: 100% 134/134 [00:00<00:00, 98.4kB/s]


## 3. Etiquetas

En los datasets CoNLL-2002/2003, hay 9 clases de tags NER:

- O, Fuera de entidad
- B-MIS, Inicio de entidad tipo miscelanea justo después de otra entidad miscelanea
- I-MIS, Entidad miscelanea
- B-PER, Inicio de entidad tipo persona justo después de otra entidad persona
- I-PER, Nombre de persona
- B-ORG, Inicio de entidad tipo organización justo después de otra entidad organización
- I-ORG, Organización
- B-LOC, Inicio de entidad tipo lugar justo después de otra entidad lugar
- I-LOC, lugar

Si existen más etiquetas en el dataset, la línea de abajo permite obtener los tags correspondientes al dataset actual. Los mismos se guardan en `labels.txt`, el cual se usará para el fine-tuning.

In [10]:
!cat train.txt dev.txt test.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

# Fine-tuning del modelo


Scripts del repositorio `transformers` a usar para el fine-tuning para NER. Luego del 21/04/2020, Hugging Face actualizó sus ejemplos para usar una nueva clase `Trainer`. Para evitar conflictos, se usa la versión anterior a estas actualizaciones.

In [11]:
#%%capture
#No usamos run_ner.py del repo original por errores con las nuevas versiones de transformers
#!wget "https://raw.githubusercontent.com/chriskhanhtran/spanish-bert/master/ner/run_ner.py"
!wget "https://raw.githubusercontent.com/chriskhanhtran/spanish-bert/master/ner/utils_ner.py"

--2022-06-10 15:32:06--  https://raw.githubusercontent.com/chriskhanhtran/spanish-bert/master/ner/utils_ner.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8603 (8.4K) [text/plain]
Saving to: ‘utils_ner.py’

utils_ner.py        100%[===================>]   8.40K  --.-KB/s    in 0s      

2022-06-10 15:32:07 (88.3 MB/s) - ‘utils_ner.py’ saved [8603/8603]



Fase de `Transfer Learning`. [Aquí](https://chriskhanhtran.github.io/posts/spanberta-bert-for-spanish-from-scratch/), se puede ver un ejemplo de pre-entrenamiento de un modelo RoBERTa en un corpus gigantesco en español, para predicción de palabras enmascaradas (masked words). Mediante este proceso se obtiene un modelo que aprendió propiedades básicas del lenguaje.

Se pueden observar abajo los hiperparámetros del modelo.

In [12]:
MAX_LENGTH = 128 #@param {type: "integer"}
MODEL = "dccuchile/bert-base-spanish-wwm-cased" #@param ["chriskhanhtran/spanberta", "bert-base-multilingual-cased", "dccuchile/bert-base-spanish-wwm-cased"]
OUTPUT_DIR = "dccuchile-ner" #@param ["spanberta-ner", "bert-base-ml-ner", "dccuchile-ner"]
BATCH_SIZE = 32 #@param {type: "integer"}
NUM_EPOCHS =  30#@param {type: "integer"}
SAVE_STEPS = 100 #@param {type: "integer"}
LOGGING_STEPS = 100 #@param {type: "integer"}
SEED = 42 #@param {type: "integer"}

Inicio del entrenamiento.

In [13]:
for i in range(30):
  SEED += i
  print(f"------------ITERACION {i}------------")

  !python3 '/content/drive/MyDrive/ProyectoNERC/NERC/run_ner.py' \
    --data_dir ./ \
    --model_type bert \
    --labels ./labels.txt \
    --model_name_or_path $MODEL \
    --output_dir $OUTPUT_DIR \
    --max_seq_length  $MAX_LENGTH \
    --num_train_epochs $NUM_EPOCHS \
    --per_gpu_train_batch_size $BATCH_SIZE \
    --save_steps $SAVE_STEPS \
    --logging_steps $LOGGING_STEPS \
    --seed $SEED \
    --iteration $i \
    --do_train \
    --do_eval \
    --do_predict \
    --overwrite_output_dir

Se han truncado las últimas 5000 líneas del flujo de salida.
Iteration:  55% 6/11 [00:04<00:03,  1.43it/s]
Iteration:  64% 7/11 [00:04<00:02,  1.43it/s]
Iteration:  73% 8/11 [00:05<00:02,  1.43it/s]
Iteration:  82% 9/11 [00:06<00:01,  1.43it/s]
Iteration:  91% 10/11 [00:07<00:00,  1.42it/s]
Iteration: 100% 11/11 [00:07<00:00,  1.49it/s]
Epoch:  70% 21/30 [02:45<01:12,  8.08s/it]
Iteration:   0% 0/11 [00:00<?, ?it/s]
Iteration:   9% 1/11 [00:00<00:07,  1.40it/s]
Iteration:  18% 2/11 [00:01<00:06,  1.41it/s]
Iteration:  27% 3/11 [00:02<00:05,  1.41it/s]
Iteration:  36% 4/11 [00:02<00:04,  1.41it/s]
Iteration:  45% 5/11 [00:03<00:04,  1.41it/s]
Iteration:  55% 6/11 [00:04<00:03,  1.41it/s]
Iteration:  64% 7/11 [00:04<00:02,  1.41it/s]
Iteration:  73% 8/11 [00:05<00:02,  1.42it/s]
Iteration:  82% 9/11 [00:06<00:01,  1.41it/s]
Iteration:  91% 10/11 [00:07<00:00,  1.41it/s]
Iteration: 100% 11/11 [00:07<00:00,  1.48it/s]
Epoch:  73% 22/30 [02:52<01:03,  7.89s/it]
Iteration:   0% 0/11 [00:00<?

In [14]:
!mv results.csv '/content/drive/MyDrive/ProyectoNERC/NERC'

Performance en el dev set:
```
21/04/2020 02:24:31 - INFO - __main__ -   ***** Eval results  *****
21/04/2020 02:24:31 - INFO - __main__ -     f1 = 0.831027443864822
21/04/2020 02:24:31 - INFO - __main__ -     loss = 0.1004064822183894
21/04/2020 02:24:31 - INFO - __main__ -     precision = 0.8207885304659498
21/04/2020 02:24:31 - INFO - __main__ -     recall = 0.8415250344510795
```
Performance en el test set:
```
21/04/2020 02:24:48 - INFO - __main__ -   ***** Eval results  *****
21/04/2020 02:24:48 - INFO - __main__ -     f1 = 0.8559533721898419
21/04/2020 02:24:48 - INFO - __main__ -     loss = 0.06848683688204177
21/04/2020 02:24:48 - INFO - __main__ -     precision = 0.845858475041141
21/04/2020 02:24:48 - INFO - __main__ -     recall = 0.8662921348314607
```

Gráficas de tensorboard del proceso de fine-tuning sobre [spanberta](https://tensorboard.dev/experiment/Ggs7aCjWQ0exU2Nbp3pPlQ/#scalars&_smoothingWeight=0.265) y [bert-base-multilingual-cased](https://tensorboard.dev/experiment/M9AXw2lORjeRzFZzEJOxkA/#scalars) para 5 épocas. Vemos que se produce overfitting luego de 3 épocas.

![](https://raw.githubusercontent.com/chriskhanhtran/spanish-bert/master/img/spanberta-ner-tb-5.JPG)





**Reporte de Clasificación**

In [ ]:
def read_examples_from_file(file_path):
    """Read words and labels from a CoNLL-2002/2003 data file.
    
    Args:
      file_path (str): path to NER data file.

    Returns:
      examples (dict): a dictionary with two keys: `words` (list of lists)
        holding words in each sequence, and `labels` (list of lists) holding
        corresponding labels.
    """
    with open(file_path, encoding="utf-8") as f:
        examples = {"words": [], "labels": []}
        words = []
        labels = []
        for line in f:
            if line.startswith("-DOCSTART-") or line == "" or line == "\n":
                if words:
                    examples["words"].append(words)
                    examples["labels"].append(labels)
                    words = []
                    labels = []
            else:
                splits = line.split(" ")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1].replace("\n", ""))
                else:
                    # Examples could have no label for mode = "test"
                    labels.append("O")
    return examples

Cargamos datos y etiquetas de los textos originales:

In [ ]:
y_true = read_examples_from_file("test.txt")["labels"]
y_pred = read_examples_from_file("spanberta-ner/test_predictions.txt")["labels"]

Reporte:

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval

print(classification_report_seqeval(y_true, y_pred))

              precision    recall  f1-score   support

        DATE       0.82      0.80      0.81        91
         LOC       0.72      0.73      0.72       223
         PER       0.68      0.70      0.69       246

   micro avg       0.72      0.73      0.72       560
   macro avg       0.74      0.74      0.74       560
weighted avg       0.72      0.73      0.73       560



Las métricas de este reporte están diseñadas específicamente para tareas NLP como NER y POS tagging, en las cuales todas las palabras de una entidad deben ser detectadas correctamente para contar como caso de éxito. Por ello los valores son más bajos que los del [reporte de clasificación de scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html).

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(np.concatenate(y_true), np.concatenate(y_pred)))

              precision    recall  f1-score   support

      B-DATE       0.94      0.83      0.88        90
       B-LOC       0.82      0.73      0.77       223
       B-PER       0.79      0.76      0.77       240
      I-DATE       0.99      0.93      0.96       206
       I-LOC       0.50      0.73      0.59        79
       I-PER       0.84      0.87      0.85       312
           O       0.99      0.99      0.99      9080

    accuracy                           0.97     10230
   macro avg       0.84      0.84      0.83     10230
weighted avg       0.97      0.97      0.97     10230

